# bioarxive Crawler
Updated 0831  
by Yejin Lee
github https://github.com/cryingjin  

>__검색 keyword와 가져올 페이지 수를 설정해줘야 합니다.__
> * keyword
> * page

In [1]:
# ready
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import urllib.request
import requests
import time as time
from selenium.common.exceptions import UnexpectedAlertPresentException
#from datetime import datetime

In [16]:
# 검색 설정
keyword = 'cancer'
page = 3

# url 불러오기
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.biorxiv.org/search/' + keyword)       # bioarxive url

In [17]:
# 페이지에 대한 크롤링
# 페이지 url을 가져오기 위함

headers = {
    'referer': 'https://www.biorxiv.org/',
    'sec-fetch-dest': 'document',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
    'Content-Type': 'application/json',
    'Cookie': '__cfduid=d5e99f8bf4299e7e0e85b6637926ca7cc1598489409; SSESS1dd6867f1a1b90340f573dcdef3076bc=MMwbZTIxrURlVCnnwvwGewJ9K3un6W3z5L4XA0d5_Ws'
}

total = []
for i in range(page):#range(3162):    페이지 수 넣어주기
    url = f'http://www.biorxiv.org/search/{keyword}?page={i}'
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    titles = [a.text for a in soup.select('span.highwire-cite-title > a > span')]
    urls = ['http://www.biorxiv.org' + a['href'] for a in soup.select('span.highwire-cite-title > a')]
    
    v = zip(titles, urls)
    total.extend(v)

#pd.DataFrame(total)

In [18]:
len(total)

30

In [19]:
# 기본 설정
urls = [total[i][1] for i in range(len(total))]

mCols = []
df = pd.DataFrame(columns=mCols)

title = []
authors = []
abstract = []
doi = []
publish_time = []

# 가져오기
for url in urls:
    
    payload = "{\"query\":\"\",\"variables\":{}}"
    headers = {
      'authority': 'www.biorxiv.org',
      'cookie': '__cfduid=dc41a60ebd7a8afcf72bb16fde51ec44e1597976999; _ga=GA1.2.1756029745.1597977006; cookie-agreed=2; SSESS1dd6867f1a1b90340f573dcdef3076bc=PPJEOPpcfuyIhMPxQldXk-6083-nS4jmcnT1LSh0y_o; has_js=1; _gid=GA1.2.1250784501.1598486250; __cfduid=d5e99f8bf4299e7e0e85b6637926ca7cc1598489409; SSESS1dd6867f1a1b90340f573dcdef3076bc=Kvu6cVf86adzYXYeTxdJ34LXr4kFU0b-4pJZWnBPaCk',
      'referer': 'https://www.biorxiv.org/search/cancer',
      'sec-fetch-dest': 'document',
      'sec-fetch-user': '?1',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
      'Content-Type': 'application/json'
    }

    response = requests.request("GET", url, headers=headers, data = payload)

    # parsing
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # paper 제목
    try:
        t = soup.select_one('#page-title')
        title.append(t.text)
    except:
        t = ''
        title.append(t)
    
    # paper 저자
    try:
        a = soup.select_one('div.highwire-cite.highwire-cite-highwire-article.highwire-citation-biorxiv-article-top.clearfix.has-author-tooltip > div.highwire-cite-authors')
        authors.append(a.text)
    except:
        a = ''
        authors.append(a)
    
    # abstract 
    try:
        ab = soup.select_one('#abstract-1')
        abstract.append(ab.text)
    except:
        ab = ''
        abstract.append(ab)    

    # paepr publishtime
    try:
        p = soup.select_one('#block-system-main > div > div > div > div > div.sidebar-right-wrapper.grid-10.omega > div > div > div.panel-pane.pane-custom.pane-1 > div')
        publish_time.append(p)
    except:
        p = ''
        publish_time.append(p.text)    
        
    # paepr doi
    try:
        d = soup.select_one('div.highwire-cite.highwire-cite-highwire-article.highwire-citation-biorxiv-article-top.clearfix.has-author-tooltip > div.highwire-cite-metadata > span')
        doi.append(d.text)
    except:
        d = ''
        doi.append(d)      

# 저장
df['title'] = title
df['authors'] = authors
df['abstract'] = abstract
df['publish_time'] = publish_time
df['doi'] = doi

,title,authors,abstract,publish_time,doi
0,Tumor Immunity Microenvironment-based classifi...,"View ORCID ProfileJialin Meng, Xiaofan Lu, Yu...",ABSTRACTBackground Bladder cancer is composed ...,"[\n Posted August 23, 2020. ]",doi: https://doi.org/10.1101/2020.08.22.262543
1,Plexin-B1 mutation drives prostate cancer meta...,"Boris Shorning, Neil Trent, David Griffiths, T...",AbstractProstate cancer mortality is associate...,"[\n Posted July 24, 2020. ]",doi: https://doi.org/10.1101/2020.07.19.203695
2,Comparative analysis of clonal evolution among...,"View ORCID ProfileSantasree Banerjee, Xianxia...",AbstractBackground Tumor multi-region sequenci...,"[\n Posted July 02, 2020. ]",doi: https://doi.org/10.1101/2020.07.01.181586
3,Cancer Classification from Healthy DNA,"Siddharth Jain, Bijan Mazaheri, Netanel Raviv,...",ABSTRACTThe genome is traditionally viewed as ...,"[\n Posted April 23, 2020. ]",doi: https://doi.org/10.1101/517839
4,Differential Mast Cell Phenotypes in Benign ve...,"Heidi Hempel Sullivan, Janielle P. Maynard, Ch...",AbstractWe previously reported that high numbe...,"[\n Posted July 24, 2020. ]",doi: https://doi.org/10.1101/2020.07.23.216408
5,Identifying Modules of Cooperating Cancer Drivers,"Michael I. Klein, Vincent L. Cannataro, Jeffre...",AbstractIdentifying cooperating modules of dri...,"[\n Posted June 29, 2020. ]",doi: https://doi.org/10.1101/2020.06.29.168229
6,Pan-cancer landscape of homologous recombinati...,"Luan Nguyen, John Martens, Arne Van Hoeck, Edw...",AbstractHomologous recombination deficiency (H...,"[\n Posted June 19, 2020. ]",doi: https://doi.org/10.1101/2020.01.13.905026
7,Prostate Epithelial Genes Define Docetaxel-Sen...,"View ORCID ProfileHyunho Han, Kwibok Choi, Yo...",ABSTRACTBACKGROUND & OBJECTIVES Analysis of th...,"[\n Posted August 28, 2020. ]",doi: https://doi.org/10.1101/2020.08.26.269415
8,Post-EMT: Cadherin-11 mediates cancer hijackin...,"Weirong Kang, Yibo Fan, Yinxiao Du, Elina A. T...",AbstractCurrent prevailing knowledge on EMT (e...,"[\n Posted August 27, 2020. ]",doi: https://doi.org/10.1101/729491
9,St. Jude Cloud—a Pediatric Cancer Genomic Data...,"Clay McLeod, Alexander M. Gout, Xin Zhou, Dela...",ABSTRACTEffective data sharing is key to accel...,"[\n Posted August 24, 2020. ]",doi: https://doi.org/10.1101/2020.08.24.264614


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         30 non-null     object
 1   authors       30 non-null     object
 2   abstract      30 non-null     object
 3   publish_time  30 non-null     object
 4   doi           30 non-null     object
dtypes: object(5)
memory usage: 1.3+ KB


In [22]:
keyword + '_doc.csv'

'cancer_doc.csv'

In [ ]:
# csv 파일로 저장
df.to_csv(keyword + '_doc.csv', index = False)